<a href="https://colab.research.google.com/github/seunghee0518/AI_python/blob/main/googlenet_with_MVtec_AD_Dataset%5Bimplemented_in_Keras%5D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

googlenet with keras
[비전 시스템을 위한 딥러닝 by moelgendy]

https://github.com/moelgendy/deep_learning_for_vision_systems/blob/master/chapter_05/inception_implementation_in_keras.ipynb

# 데이터 셋 전처리 및 시각화

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
train_dir = '/content/drive/MyDrive/DACON_이상치 탐지 알고리즘 경진대회/open/train'
valid_dir = '/content/drive/MyDrive/DACON_이상치 탐지 알고리즘 경진대회/open/test'

train_df = pd.read_csv('/content/drive/MyDrive/DACON_이상치 탐지 알고리즘 경진대회/open/train_df.csv')
test_df = pd.read_csv('/content/drive/MyDrive/DACON_이상치 탐지 알고리즘 경진대회/open/test_df.csv')

In [20]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import keras
import os # miscellaneous operating system interfaces
import shutil # high-level file operations

In [30]:
base_dir = '/content/drive/MyDrive/DACON_이상치 탐지 알고리즘 경진대회/open/'
train_dir = '/content/drive/MyDrive/DACON_이상치 탐지 알고리즘 경진대회/open/train'
test_dir = '/content/drive/MyDrive/DACON_이상치 탐지 알고리즘 경진대회/open/test'

train_df = pd.read_csv('/content/drive/MyDrive/DACON_이상치 탐지 알고리즘 경진대회/open/train_df.csv')
test_df = pd.read_csv('/content/drive/MyDrive/DACON_이상치 탐지 알고리즘 경진대회/open/test_df.csv')

In [23]:
len(os.listdir(train_dir)), len(os.listdir(test_dir))

(4277, 2154)

In [28]:
## train 데이터 개수 및 레이블 개수
train_label = train_df.label.tolist()
train_label_unique = list(set(train_label))
print(len(train_label), len(train_label_unique))

4277 88


In [38]:
## 88개 레이블 폴더 생성
os.mkdir(base_dir+f'train_label_88')
for dir in train_label_unique:
  os.mkdir(base_dir+f'train_label_88/{dir}')

In [37]:
train_df

,index,file_name,class,state,label
0,0,10000.png,transistor,good,transistor-good
1,1,10001.png,capsule,good,capsule-good
2,2,10002.png,transistor,good,transistor-good
3,3,10003.png,wood,good,wood-good
4,4,10004.png,bottle,good,bottle-good
...,...,...,...,...,...
4272,4272,14272.png,transistor,good,transistor-good
4273,4273,14273.png,transistor,good,transistor-good
4274,4274,14274.png,grid,good,grid-good
4275,4275,14275.png,zipper,good,zipper-good


In [40]:
import shutil
from tqdm import tqdm

for i in tqdm(range(len(train_df))):
  file_name = train_df.loc[i, 'file_name']
  label     = train_df.loc[i, 'label']

  source = train_dir + f'/{file_name}'
  destination = base_dir + f'train_label_88/{label}/{file_name}'
  shutil.copyfile(source, destination) # 이동이 아닌 카피로 실행함
  # shutil.move(file_source, file_destination) # 이동하는 코드

100%|██████████| 4277/4277 [23:58<00:00,  2.97it/s]


In [48]:
# 분리된 이미지 파일 전체 개수 확인
all_folder = glob(base_dir + 'train_label_88/*')
all_img_cnt = 0
all_img_path = []
for folder in tqdm(all_folder, total = len(all_folder)):
  img_list = glob(folder + '/*')
  all_img_path += img_list
  img_cnt = len(img_list)
  all_img_cnt += img_cnt

100%|██████████| 88/88 [00:00<00:00, 417.76it/s]


In [49]:
print('분리된 이미지 파일 총 개수 : ', all_img_cnt)

분리된 이미지 파일 총 개수 :  4277


## 이미지 파일 로딩 + float array 변환 + 전처리

In [53]:
from keras.preprocessing import image

def preprocess_img(img_path, target_size):
  img = image.load_img(img_path, target_size = (target_size, target_size))
  img_tensor = image.img_to_array(img)
  img_tensor = np.expand_dims(img_tensor, axis = 0) # 차원 추가
  img_tensor /= 255. # [0, 1] 범위로 표준화
  return img_tensor

In [55]:
## 이미지 시각화(랜덤 30개만)
import random
n_pic = 30 # 시각화할 이미지 수
n_col = 5 # 가로 개수
n_row = 6 # 세로 개수

target_size = 100
margin = 3

total = np.zeros((n_row * target_size + (n_row - 1) * margin, n_col * target_size + (n_col - 1) * margin, 3))

img_seq = 0

# 전체 중 랜덤 샘플 추출함
sampleList = random.sample(all_img_path, n_pic)

for i in range(n_row):
  for j in range(n_col):
    img_path = sampleList[img_seq]
    img_tensor = preprocess_img(img_path, target_size)

    h_start = i * target_size + i * margin
    h_end   = h_start + target_size
    v_start = j * target_size + j * margin
    v_end   = v_start + target_size

    total[h_start:h_end, v_start:v_end] = img_tensor[0]
    img_seq += 1

plt.figure(figsize = (200, 200))
plt.imshow(total)
plt.show()

## 이미지 파일 -> 학습 데이터로 변경

In [ ]:
# 학습 파일 기본 경로 지정 및 분류할 클래스 
import re
import cv2
from sklearn.model_selection import train_test_split

# base_dir : /content/drive/MyDrive/DACON_이상치 탐지 알고리즘 경진대회/open/
train_base_dir = base_dir + '/train_label_88/'

categories = train_label_unique
categories.sort() # 알파벳 순으로 정렬
num_classes = len(categories)

# 이미지 사이즈 변경 : 224*224
img_w = 224
img_h = 224

X = [] # 입력 데이터
Y = [] # 정답값

for idx, categorie in enumerate(categories):
  ## label -> 원-핫 인코딩
  label = [0 for i in range(num_classes)]
  label[idx] = 1

  # 이미지 파일 경로
  image_dir = train_base_dir + categorie + '/'

  # 레이블 폴더 별로 이미지 파일 추출
  for top, dir, f in os.walk(image_dir):
    for filename in f:
      img = cv2.imread(image_dir + filename)
      img = cv2.resize(img, None, fx=img_w/img.shape[1], fy=img_h/img.shape[0])
      X.append(img/256)
      Y.append(label)

# 배열 -> 넘파이 배열로 변환
X = np.array(X)
Y = np.array(Y)

# 학습 / 검증 데이터로 분리
X_train, X_test, y_train, y_test = train_test_split(X, Y)

X_train = X_train.astype('float32')
y_train = y_train.astype('float32')

#numpy 배열 저장하기
xy = (X_train, X_test, y_train, y_test)
np.save(f"{train_base_dir}img_data.npy", xy)

In [ ]:
X_train

1. Import libraries

In [ ]:
#!pip install keras

In [71]:
import cv2 
import numpy as np 
import pandas as pd
import keras
import math
#from tensorflow import keras
from tensorflow.keras import layers
#from keras.layers.core import Layer # !pip install dependencies # 설치 안됨
import keras.backend as K
import tensorflow as tf
from keras.datasets import cifar10
from keras.models import Model
from keras.layers import Conv2D, MaxPool2D,  \
    Dropout, Dense, Input, concatenate,      \
    GlobalAveragePooling2D, AveragePooling2D,\
    Flatten
#from keras.datasets import cifar10 
from keras import backend as K 
from keras.utils import np_utils 
from tensorflow.keras.optimizers import SGD 
#from keras.optimizers import SGD 
from keras.callbacks import LearningRateScheduler

from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from glob import glob

In [72]:
print(keras.__version__)

2.8.0


2. Prepare the dataset

In [ ]:
# #num_classes = 10 # 분류할 레이블 수

# def load_cifar10_data(img_rows, img_cols):

#     # Load cifar10 training and validation sets
#     (X_train, Y_train), (X_valid, Y_valid) = cifar10.load_data()

#     # Resize training images
#     X_train = np.array([cv2.resize(img, (img_rows,img_cols)) for img in X_train[:,:,:,:]])
#     X_valid = np.array([cv2.resize(img, (img_rows,img_cols)) for img in X_valid[:,:,:,:]])

#     # Transform targets to keras compatible format [np_utils.to_categorical ->  Converts a class vector (integers) to binary class matrix] 
#     Y_train = np_utils.to_categorical(Y_train, num_classes)
#     Y_valid = np_utils.to_categorical(Y_valid, num_classes)
    
#     X_train = X_train.astype('float32')
#     X_valid = X_valid.astype('float32')

#     # preprocess data
#     X_train = X_train / 255.0
#     X_valid = X_valid / 255.0

#     return X_train, Y_train, X_valid, Y_valid


In [ ]:
# num_classes = 10 : 분류할 레이블 수
#X_train, y_train, X_test, y_test = load_cifar10_data(224, 224)

3. Create the inception module

In [73]:
def inception_module(x,
                     filters_1x1,
                     filters_3x3_reduce,
                     filters_3x3,
                     filters_5x5_reduce,
                     filters_5x5,
                     filters_pool_proj,
                     name=None):
    
    conv_1x1 = Conv2D(filters_1x1, (1, 1), padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(x)
    
    conv_3x3 = Conv2D(filters_3x3_reduce, (1, 1), padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(x)
    conv_3x3 = Conv2D(filters_3x3, (3, 3), padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(conv_3x3)

    conv_5x5 = Conv2D(filters_5x5_reduce, (1, 1), padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(x)
    conv_5x5 = Conv2D(filters_5x5, (5, 5), padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(conv_5x5)

    pool_proj = MaxPool2D((3, 3), strides=(1, 1), padding='same')(x)
    pool_proj = Conv2D(filters_pool_proj, (1, 1), padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(pool_proj)

    output = concatenate([conv_1x1, conv_3x3, conv_5x5, pool_proj], axis=3, name=name)
    ## concatenate  : 선택한 축 방향으로 배열 연결하는 함수[3차원 일 경우 0 : 높이, 1 : 행, 2 : 열, 3 : ?]

    return output

4. Create the GoogLeNet architecture

In [74]:
kernel_init = keras.initializers.glorot_uniform()
bias_init = keras.initializers.Constant(value=0.2)

In [75]:
input_layer = Input(shape=(224, 224, 3))

x = Conv2D(64, (7, 7), padding='same', strides=(2, 2), activation='relu', name='conv_1_7x7/2', kernel_initializer=kernel_init, bias_initializer=bias_init)(input_layer)
x = MaxPool2D((3, 3), padding='same', strides=(2, 2), name='max_pool_1_3x3/2')(x)
# x = Conv2D(64, (1, 1), padding='same', strides=(1, 1), activation='relu', name='conv_2a_3x3/1')(x)
x = Conv2D(192, (3, 3), padding='same', strides=(1, 1), activation='relu', name='conv_2b_3x3/1')(x)
x = MaxPool2D((3, 3), padding='same', strides=(2, 2), name='max_pool_2_3x3/2')(x)

x = inception_module(x,
                     filters_1x1=64,
                     filters_3x3_reduce=96,
                     filters_3x3=128,
                     filters_5x5_reduce=16,
                     filters_5x5=32,
                     filters_pool_proj=32,
                     name='inception_3a')

x = inception_module(x,
                     filters_1x1=128,
                     filters_3x3_reduce=128,
                     filters_3x3=192,
                     filters_5x5_reduce=32,
                     filters_5x5=96,
                     filters_pool_proj=64,
                     name='inception_3b')

x = MaxPool2D((3, 3), padding='same', strides=(2, 2), name='max_pool_3_3x3/2')(x)

x = inception_module(x,
                     filters_1x1=192,
                     filters_3x3_reduce=96,
                     filters_3x3=208,
                     filters_5x5_reduce=16,
                     filters_5x5=48,
                     filters_pool_proj=64,
                     name='inception_4a')


classifier_1 = AveragePooling2D((5, 5), strides=3)(x)
classifier_1 = Conv2D(128, (1, 1), padding='same', activation='relu')(classifier_1)
classifier_1 = Flatten()(classifier_1)
classifier_1 = Dense(1024, activation='relu')(classifier_1)
classifier_1 = Dropout(0.7)(classifier_1)
classifier_1 = Dense(10, activation='softmax', name='auxilliary_output_1')(classifier_1)

x = inception_module(x,
                     filters_1x1=160,
                     filters_3x3_reduce=112,
                     filters_3x3=224,
                     filters_5x5_reduce=24,
                     filters_5x5=64,
                     filters_pool_proj=64,
                     name='inception_4b')

x = inception_module(x,
                     filters_1x1=128,
                     filters_3x3_reduce=128,
                     filters_3x3=256,
                     filters_5x5_reduce=24,
                     filters_5x5=64,
                     filters_pool_proj=64,
                     name='inception_4c')

x = inception_module(x,
                     filters_1x1=112,
                     filters_3x3_reduce=144,
                     filters_3x3=288,
                     filters_5x5_reduce=32,
                     filters_5x5=64,
                     filters_pool_proj=64,
                     name='inception_4d')


classifier_2 = AveragePooling2D((5, 5), strides=3)(x)
classifier_2 = Conv2D(128, (1, 1), padding='same', activation='relu')(classifier_2)
classifier_2 = Flatten()(classifier_2)
classifier_2 = Dense(1024, activation='relu')(classifier_2)
classifier_2 = Dropout(0.7)(classifier_2)
classifier_2 = Dense(10, activation='softmax', name='auxilliary_output_2')(classifier_2)

x = inception_module(x,
                     filters_1x1=256,
                     filters_3x3_reduce=160,
                     filters_3x3=320,
                     filters_5x5_reduce=32,
                     filters_5x5=128,
                     filters_pool_proj=128,
                     name='inception_4e')

x = MaxPool2D((3, 3), padding='same', strides=(2, 2), name='max_pool_4_3x3/2')(x)

x = inception_module(x,
                     filters_1x1=256,
                     filters_3x3_reduce=160,
                     filters_3x3=320,
                     filters_5x5_reduce=32,
                     filters_5x5=128,
                     filters_pool_proj=128,
                     name='inception_5a')

x = inception_module(x,
                     filters_1x1=384,
                     filters_3x3_reduce=192,
                     filters_3x3=384,
                     filters_5x5_reduce=48,
                     filters_5x5=128,
                     filters_pool_proj=128,
                     name='inception_5b')

x = AveragePooling2D(pool_size=(7,7), strides=1, padding='valid',name='avg_pool_5_3x3/1')(x)

x = Dropout(0.4)(x)
x = Dense(1000, activation='relu', name='linear')(x)
x = Dense(1000, activation='softmax', name='output')(x)

GoogleNet model without the classfiers 1 and 2

In [76]:
model = Model(input_layer, [x], name='googlenet')

In [77]:
model.summary()

Model: "googlenet"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv_1_7x7/2 (Conv2D)          (None, 112, 112, 64  9472        ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 max_pool_1_3x3/2 (MaxPooling2D  (None, 56, 56, 64)  0           ['conv_1_7x7/2[0][0]']           
 )                                                                                        

GoogLeNet model after adding classifiers 1 and 2

In [78]:
model_with_classifiers = Model(input_layer, [x, classifier_1, classifier_2], name='googlenet_complete_architecture')
model_with_classifiers.summary()

Model: "googlenet_complete_architecture"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv_1_7x7/2 (Conv2D)          (None, 112, 112, 64  9472        ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 max_pool_1_3x3/2 (MaxPooling2D  (None, 56, 56, 64)  0           ['conv_1_7x7/2[0][0]']           
 )                                                                  

compile a network with three models

In [ ]:
epochs = 25
initial_lrate = 0.01

def decay(epoch, steps=100):
    initial_lrate = 0.01
    drop = 0.96
    epochs_drop = 8
    lrate = initial_lrate * math.pow(drop, math.floor((1+epoch)/epochs_drop))
    return lrate

sgd = SGD(lr=initial_lrate, momentum=0.9, nesterov=False)

lr_sc = LearningRateScheduler(decay, verbose=1)

model_with_classifiers.compile(loss=['categorical_crossentropy', 'categorical_crossentropy', 'categorical_crossentropy'], 
              loss_weights=[1, 0.3, 0.3], optimizer=sgd, metrics=['accuracy'])

history = model_with_classifiers.fit(X_train, [y_train, y_train, y_train], validation_data=(X_test, [y_test, y_test, y_test]), 
                    epochs=epochs, batch_size=256, callbacks=[lr_sc])